In [14]:
import logging

def get_logger_name(name=None):
    root_name = 'notmnist.model'
    return root_name if name is None else root_name + '.' + name

def setup_logging():
    logger = logging.getLogger('notmnist.model')
    logger.setLevel(logging.DEBUG)
    ch = logging.StreamHandler()
    fh = logging.FileHandler('notmnist-model.log')
    ch.setLevel(logging.DEBUG)
    fh.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(asctime)s [%(threadName)s] - %(name)s - %(levelname)s - %(message)s')
    ch.setFormatter(formatter)
    fh.setFormatter(formatter)
    logger.addHandler(ch)
    logger.addHandler(fh)

setup_logging()

In [54]:
import os
import sys
import tarfile
import glob
import tensorflow as tf
import numpy as np
from urllib.request import urlretrieve

# 'http://yaroslavvb.com/upload/notMNIST/'

class notMNISTDataset(object):
    def __init__(self, url, filename, num_classes, force=False):
        self.log = logging.getLogger(get_logger_name(self.__class__.__name__))
        self.url = url
        self.num_classes = num_classes
        self.filename = filename
        
        self._maybe_download(force)
        self._maybe_extract(force)
        
    def _maybe_download(self, force=False):
        """Download a file if not present, and make sure it's the right size."""
        
        def download_progress(count, blockSize, totalSize):
            percent = int(count * blockSize * 100 / totalSize)
            
            if count % 500 == 0:
                self.log.info("%2d%%" % percent)
        
        if force or not os.path.exists(self.filename):
            url = self.url + self.filename
            self.log.info("Downloading %s" % url)
            self.filename, _ = urlretrieve(url, self.filename, reporthook=download_progress)
    
    def _maybe_extract(self, force=False):
        self._data_folder = os.path.splitext(os.path.splitext(self.filename)[0])[0]  # remove .tar.gz
        
        if os.path.isdir(self._data_folder) and not force:
            # You may override by setting force=True.
            self.log.info('%s already present - Skipping extraction of %s.' % (self._data_folder, self.filename))
        else:
            self.log.info('Extracting data for %s. This may take a while. Please wait.' % self._data_folder)
            
            self._extract_tar(self.filename)
            
            self._data_folders = [os.path.join(root, d) 
                                  for d in sorted(os.listdir(root))
                                  if os.path.isdir(os.path.join(root, d))]
            
            if len(self._data_folders) != self.num_classes:
                raise Exception(
                  'Expected %d folders, one per class. Found %d instead.' % (
                    self.num_classes, len(self._data_folders)))
                
            self.log.info(self._data_folders)
            
    def read_batch(self, batch_size=32, num_threads=1):
        self.log.info("Preparing data reader")
        self.filenames = glob.glob(self._data_folder + '/**/*.png', recursive=True)
        self.labels = [f.split(os.sep)[1] for f in self.filenames]
        
        labels_enc = np.array([ord(x) - 65 for x in self.labels])
        labels_enc = notMNISTDataset._one_hot(labels_enc)
        filename_queue = tf.train.string_input_producer(self.filenames, name='filename_queue')
        label_fifo = tf.FIFOQueue(len(self.labels), tf.float32, shapes=[labels_enc.shape[1:]], name='label_fifo')
        label_enqueue = label_fifo.enqueue_many([labels_enc])
        
        # lets add label queue runner to graph collection so it would start with other queue runners
        label_qr = tf.train.QueueRunner(label_fifo, [label_enqueue])
        tf.add_to_collection(tf.GraphKeys.QUEUE_RUNNERS, label_qr)
        
        data = self._load_png(filename_queue)
        label = label_fifo.dequeue()
        
        self.log.info("Reading batch")
        data_batch, label_batch = tf.train.shuffle_batch([data, label],
                                                          batch_size=batch_size,
                                                          num_threads=num_threads,
                                                          capacity=batch_size * 2,
                                                          min_after_dequeue=0,
                                                          allow_smaller_final_batch=True, 
                                                          name='batch_queue')
        
        
        tf.summary.image('image batch', data_batch)
        
        # return images, tf.reshape(label_batch, [batch_size])
        return data_batch, label_batch
        
    @staticmethod
    def _load_png(input_queue):
        reader = tf.WholeFileReader()
        
        _, data = reader.read(input_queue)
        data = tf.image.decode_png(data)
        data.set_shape([28, 28, 1])
        return data

    @staticmethod
    def _extract_tar(filename):
        tar = tarfile.open(filename)
        sys.stdout.flush()
        tar.extractall()
        tar.close()
        
    @staticmethod
    def _one_hot(array):
        b = np.zeros((array.size, array.max() + 1))
        b[np.arange(array.size), array] = 1
        return b

In [60]:
import tensorflow as tf

def weights(shape):
    w = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(w)

def bias(shape):
    b = tf.constant(0.1, shape=shape)
    return tf.Variable(b)

def conv2d(x, w):
    return tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='SAME')

def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def conv2d_layer(layer_input, filter_shape):
    w = weights(filter_shape)
    b = bias([filter_shape[-1]])
    return tf.nn.relu(conv2d(layer_input, w) + b)

def dense_layer(layer_input, filter_shape, activation='relu'):
    w = weights(filter_shape)
    b = bias([filter_shape[-1]])
    linear = tf.matmul(layer_input, w) + b
    
    if activation == 'relu':
        return tf.nn.relu(linear)
    elif activation is None:
        return linear
    else:
        raise ValueError('activation %s not supported' % activation)
        
def simple_convnet(data_batch):
    images = tf.to_float(tf.reshape(data_batch, [-1, 28, 28, 1]))
    c1 = conv2d_layer(images, [5, 5, 1, 32])
    mp1 = max_pool(c1)

    c2 = conv2d_layer(mp1, [5, 5, 32, 64])
    mp2 = max_pool(c2)

    mp2 = tf.reshape(mp2, [-1, 7 * 7 * 64])
    d3 = dense_layer(mp2, [7 * 7 * 64, 1024])
    d3 = tf.nn.dropout(d3, 0.3)

    y_out = dense_layer(d3, [1024, 10], activation=None)
    
    return y_out

In [ ]:
import tensorflow as tf
import logging

tf.reset_default_graph()
config = tf.ConfigProto()
config.operation_timeout_in_ms=100000

# load data and prepare ops
train_dataset = notMNISTDataset(url='http://yaroslavvb.com/upload/notMNIST/', 
                    filename='notMNIST_large.tar.gz',
                   num_classes=10)

# create network and optimizer
data_batch, label_batch = train_dataset.read_batch()
y_out =  simple_convnet(data_batch)

error = tf.nn.softmax_cross_entropy_with_logits(y_out, label_batch)
train_step = tf.train.AdamOptimizer(1e-4).minimize(error)

correct_prediction = tf.equal(tf.argmax(y_out, 1), tf.argmax(label_batch, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar("training_accuracy", accuracy)

summary_writer = tf.summary.FileWriter('.')
merged = tf.summary.merge_all()

# run training                                      
with tf.Session(config=config) as sess:
    print("Starting session")
    sess.run(tf.global_variables_initializer())
    
    # start queues
    coordinator = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coordinator)
    
    while not coordinator.should_stop():  
        print("Running batch reading op")
        train_step.run()
        
        # write summaries for Tensorboard
        summary = sess.run(merged)
        summary_writer.add_summary(summary)
        summary_writer.add_graph(sess.graph)

    print("Requesting stop...")
    coordinator.request_stop()
    coordinator.join(threads)

2017-01-18 17:07:45,450 [MainThread] - notmnist.model.notMNISTDataset - INFO - notMNIST_large already present - Skipping extraction of notMNIST_large.tar.gz.
2017-01-18 17:07:45,452 [MainThread] - notmnist.model.notMNISTDataset - INFO - Preparing data reader
2017-01-18 17:07:54,030 [MainThread] - notmnist.model.notMNISTDataset - INFO - Reading batch


Instructions for updating:
Please switch to tf.summary.image. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, the max_images argument was renamed to max_outputs.
Starting session
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Running batch reading op
Runni